In [10]:
import pandas as pd

In [11]:
train_df = pd.read_parquet('/home/user/wyf/deepseek_learn/deepseek_r1_train/gsm8k_chinese/data/train-00000-of-00001.parquet')
test_df = pd.read_parquet('/home/user/wyf/deepseek_learn/deepseek_r1_train/gsm8k_chinese/data/test-00000-of-00001.parquet')

In [12]:
train_df.head()

,question,answer,question_zh-cn,answer_only
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,纳塔利娅在 4 月份向 48 个朋友出售了视频片段，然后在 5 月份售出了一半的视频片段。娜...,72
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,翁靠做保姆每小时挣 12 美元。昨天，她只做了 50 分钟的保姆工作。她赚了多少钱？,10
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...",贝蒂正在攒钱买一个价值 100 美元的新钱包。贝蒂只有她需要的一半钱。她的父母决定为此给她 ...,5
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,朱莉正在读一本 120 页的书。昨天，她能读12页，今天，她读的页数是昨天的两倍。如果她明天...,42
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,詹姆斯每周两次给两个不同的朋友写一封三页的信。他一年写多少页？,624


In [13]:
system_prompt = [
    {
        'role': 'system', 
        'content': (
    "Solve the following problem step by step.\n"
    "First, provide your reasoning in <think> and </think>\n"
    "After reasoning, provide your final answer in <answer> and </answer>\n"
)
    }
    ]

In [ ]:
def process_data(df, mode):
    data_source = []
    prompt = []
    ability = []
    reward_model = []
    extra_info = []
    
    for index, row in df.iterrows():
        data_source.append('math')
        p = system_prompt + [{"role": "user", "content": row['question_zh-cn']}]
        prompt.append(p)
        ability.append('math')
        reward_model.append({
                    "style": "rule",
                    "ground_truth": row['answer_only'],
                })
        
        extra_info.append({
                    'split': mode,
                    'index': index,
                })
    df['data_source'] = data_source
    df['prompt'] = prompt
    df['ability'] = ability
    df['reward_model'] = reward_model
    df['extra_info'] = extra_info
    
    return df

    

In [16]:
train_data = process_data(train_df, 'train')
test_data = process_data(test_df, 'test')
train_data.to_parquet('math_train.parquet', index=False)
test_data.to_parquet('math_test.parquet', index=False)